In [ ]:
repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir
from spf.dataset.spf_dataset import SessionsDatasetReal

In [ ]:
from spf.dataset.spf_dataset import SessionsDatasetRealV3Rx, pi_norm


data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/rover_data/march9"
data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/debug_data/march9/1"
data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march13/roof/rover1_recording_straight_line"
data_root_dir = (
    "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march13/rover3/second_try"
)
data_root_dir = (
    "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march13/rover1/second_try"
)
# data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march14/rover3"
# data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march13/roof/laptop_recording"

# march 17
data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march17/rover3"
# data_root_dir = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march17/rover2/1"

width = 3000  # height is assumed to be same
step_size = 1

ds = SessionsDatasetRealV3Rx(
    root_dir=data_root_dir,
    snapshots_in_session=4000,  # 2**15
    nsources=1,
    step_size=step_size,
    check_files=False,
)

# pick a session from the dataset
session_idx = 0  # 2**14
session1 = ds[(0, session_idx)]
# session2 = session1
session2 = ds[(1, session_idx)]
print(ds.root_dir)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import datetime

z = datetime.datetime.fromtimestamp(session2["time_stamps"][0])  #
# .strftime(
#     "%Y-%m-%d %H:%M:%S"
# )
z.minute, z

In [ ]:
z = datetime.datetime.fromtimestamp(session2["time_stamps"][0])  #
z

In [ ]:
z = datetime.datetime.fromtimestamp(session2["time_stamps"][0])  #
z

In [ ]:
z = datetime.datetime.fromtimestamp(datetime.datetime.now().timestamp())
z.strftime("%Y_%m_%d_%H_%M_%S")

In [ ]:
z = datetime.datetime.fromtimestamp(datetime.datetime.now().timestamp())
z.strftime("%Y_%m_%d_%H_%M_%S")

In [ ]:
# from spf.wall_array_v2 import v3_column_names

# rows = 600000
# col_names = v3_column_names()
# cols = len(col_names)
# fn = data_root_dir + "/rover_2024_03_09_07_45_16_nRX2_circle.npy"
# m = np.memmap(fn, dtype="float32", mode="r", shape=(rows, cols))

In [ ]:
import numpy


def smooth(x, window_len=11, window="hanning"):
    """smooth the data using a window with requested size.

    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.

    input:
        x: the input signal
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal

    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)

    see also:

    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter

    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """
    if x.ndim != 1:
        raise ValueError  # , "smooth only accepts 1 dimension arrays."
    if x.size < window_len:
        raise ValueError  # , "Input vector needs to be bigger than window size."
    if window_len < 3:
        return x
    if not window in ["flat", "hanning", "hamming", "bartlett", "blackman"]:
        raise ValueError  # , "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
    s = numpy.r_[x[window_len - 1 : 0 : -1], x, x[-2 : -window_len - 1 : -1]]
    # print(len(s))
    if window == "flat":  # moving average
        w = numpy.ones(window_len, "d")
    else:
        w = eval("numpy." + window + "(window_len)")
    y = numpy.convolve(w / w.sum(), s, mode="valid")
    return y

In [ ]:
session["beam_former_outputs_at_t"][:, 0].shape
range(session["beam_former_outputs_at_t"].shape[1])

In [ ]:
smoothed = np.vstack(
    [
        smooth(session["beam_former_outputs_at_t"][:, x], window_len=128)
        for x in range(session["beam_former_outputs_at_t"].shape[1])
    ]
).T
peaks_at_t = np.array([get_peaks_for_2rx(bf_out)[:2] for bf_out in smoothed])
plt.plot(peaks_at_t[:, 0])

In [ ]:
from spf.rf import get_peaks_for_2rx

session = session2
window = "flat"
window_len = 128


peaks_at_t = np.array(
    [get_peaks_for_2rx(bf_out)[:2] for bf_out in session["beam_former_outputs_at_t"]]
)
peaks_at_t_in_radians = (
    # beam former outputs are an array with -pi,...,+pi
    2
    * (peaks_at_t / session["beam_former_outputs_at_t"].shape[1] - 0.5)
    * np.pi
)
plt.plot(smooth(peaks_at_t_in_radians[:, 0], window_len=window_len, window=window))
plt.plot(smooth(peaks_at_t_in_radians[:, 1], window_len=window_len, window=window))

In [ ]:
session1["gain"].mean()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spf.rf import get_peaks_for_2rx

# print("IDX", idx, len(session))
idx = 0  # session["beam_former_outputs_at_t"].shape[0] // 3
bf_out = session1["beam_former_outputs_at_t"][idx + 1]
peaks = get_peaks_for_2rx(bf_out)
plt.plot(bf_out, color="red", label="beamformer")
for peak in peaks:
    plt.axvline(x=peak, label="Peak")

plt.legend()
peaks

In [ ]:
session1.keys()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sessions = [session1, session2]
print("TODO SESSIONS ARE IDENTICAL!!!")
f, axs = plt.subplots(3, 1 + len(sessions), figsize=(8 * (1 + len(sessions)), 4 * 3))

axs[0, 0].invert_yaxis()  # y goes down
axs[0, 0].invert_xaxis()  # x goes left
axs[0, 0].set_xlabel("X position")
axs[0, 0].set_ylabel("Y position")
# axs[0, 0].scatter(
#    session1["source_positions_at_t"][:, 0, 0],
#    session1["source_positions_at_t"][:, 0, 1],
#    label="TX0",
# )
axs[0, 0].scatter(
    session1["detector_position_at_t"][:, 0],
    session1["detector_position_at_t"][:, 1],
    label="RX0",
)
axs[0, 0].set_title("2D Position (x/y)")
axs[0, 0].legend()

# axs[1, 0].plot(session1["source_positions_at_t"][:, 0, 0], label="source X")
# axs[1, 0].plot(session1["source_positions_at_t"][:, 0, 1], label="source Y")
axs[1, 0].plot(session1["detector_position_at_t"][:, 0], label="receiver X")
axs[1, 0].plot(session1["detector_position_at_t"][:, 1], label="receiver Y")
axs[1, 0].legend()
axs[1, 0].set_title("Position (x/y) vs time")
axs[1, 0].set_xlabel("Time")

# TODO THERE IS A BUG HERE? SHOULDNT THE ANGLES BE WIDER?
for session_idx in range(len(sessions)):
    session = sessions[session_idx]
    # axs[1, 1 + session_idx].plot(pi_norm(session["source_theta_at_t"]))
    axs[1, 1 + session_idx].set_ylabel("Theta")
    axs[1, 1 + session_idx].set_xlabel("Time")
    axs[1, 1 + session_idx].set_title("Ground truth theta")
    axs[1, 1 + session_idx].set_ylim([-np.pi, +np.pi])

    # plot the beamformer output
    peaks_at_t = np.array(
        [
            get_peaks_for_2rx(bf_out)[:2]
            for bf_out in session["beam_former_outputs_at_t"]
        ]
    )
    peaks_at_t_in_radians = (
        # beam former outputs are an array with -pi,...,+pi
        2
        * (peaks_at_t / session["beam_former_outputs_at_t"].shape[1] - 0.5)
        * np.pi
    )
    # peaks_at_t_in_radians_adjusted = pi_norm(  # spf dataset already adjusts for us
    #    peaks_at_t_in_radians  # + session["detector_orientation_at_t"]
    # )
    axs[0, 1 + session_idx].set_title(f"SDR:{session_idx} phaseRX0-phaseRX1")
    axs[0, 1 + session_idx].scatter(
        np.arange(peaks_at_t_in_radians.shape[0]),
        peaks_at_t_in_radians[:, 0],
        alpha=0.1,
    )
    axs[0, 1 + session_idx].scatter(
        np.arange(peaks_at_t_in_radians.shape[0]),
        peaks_at_t_in_radians[:, 1],
        alpha=0.1,
    )
    axs[0, 1 + session_idx].set_ylim([-np.pi, +np.pi])
    axs[0, 1 + session_idx].set_xlabel("Time")
    axs[0, 1 + session_idx].set_ylabel("Theta")

    def norm(x):
        return (x - x.mean()) / x.std()

    axs[2, 1 + session_idx].plot(
        norm(session["avg_phase_diffs"][:, 0]), label="avg phase diff"
    )
    axs[2, 1 + session_idx].plot(norm(session["rssi"]), label="rssi")
    axs[2, 1 + session_idx].plot(norm(session["other_gain"]), label="other_gain")
    axs[2, 1 + session_idx].plot(norm(session["gain"]), label="gain")
    axs[2, 1 + session_idx].legend()
    axs[2, 1].set_title("Z-normed rssi and gain")

f.tight_layout()

In [ ]:
import numpy as np


def a_to_b_in_stepsize(a, b, step_size):
    if np.isclose(a, b).all():
        return [b]
    # move by step_size from where we are now to the target position
    points = []
    direction = (b - a) / np.linalg.norm(b - a)
    distance = np.linalg.norm(b - a)
    _l = step_size
    while _l < distance:
        points.append(_l * direction + a)
        _l += step_size
    points.append(b)
    return points


def a_to_b_in_stepsize_np(a, b, step_size):
    if np.isclose(a, b).all():
        return [b]
    # move by step_size from where we are now to the target position
    distance = np.linalg.norm(b - a)
    steps = np.arange(1, np.ceil(distance / step_size) + 1) * step_size

    direction = (b - a) / np.linalg.norm(b - a)
    points = a.reshape(1, 2) + direction.reshape(1, 2) * steps.reshape(len(steps), 1)
    points[-1] = b
    return points


import time

st = time.time()
for _ in range(100):
    a_to_b_in_stepsize(np.array([10, 10]), np.array([20, 20]), 0.01)
print(time.time() - st)

In [ ]:
import time

st = time.time()
for _ in range(100):
    a_to_b_in_stepsize_np(np.array([10, 10]), np.array([20, 20]), 0.01)
print(time.time() - st)

In [ ]:
len(a_to_b_in_stepsize(np.array([10, 10]), np.array([20, 20]), 0.01))

In [ ]:
len(a_to_b_in_stepsize_np(np.array([10, 10]), np.array([20, 20]), 0.01))

In [ ]:
    [300, 400],
    [3100, 400],
    [3100, 3200],
    [2000, 3000],
    # [300,1500],
    [800, 1500],
    [300, 500]

In [ ]:
import glob
import os

import numpy as np

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir
from spf.dataset.spf_dataset import SessionsDatasetReal

from spf.dataset.spf_dataset import SessionsDatasetRealV2

directory = "/Users/miskodzamba/Dropbox/research/gits/spf/spf/v2_data/v2.1"

list_of_files = glob.glob(f"{directory}/*.npy")
latest_file = max(list_of_files, key=os.path.getctime)
print(latest_file)

root_dir = directory
ds = SessionsDatasetRealV2(
    root_dir=root_dir,
    snapshots_in_session=-1,
    check_files=False,
    step_size=1,
    filenames=[latest_file],
)

bfo = ds[0]["beam_former_outputs_at_t"].copy()
bfo[-10:] = 0  # REMOVE THIS IS TESTING!!
print("TESTING")
if (bfo == 0).min(axis=1).max():
    first_invalid_idx = np.argmax((bfo == 0).min(axis=1))
    assert first_invalid_idx != 0
    last_valid_idx = first_invalid_idx - 1

    # get receivers
    rx0 = ds[0]["receiver_positions_at_t"][last_valid_idx].mean(axis=0)
    rx1 = ds[1]["receiver_positions_at_t"][last_valid_idx].mean(axis=0)
    assert np.isclose(rx0, rx1).all()

    # get the emitter
    tx0 = ds[1]["source_positions_at_t"][-1, 0]

    print(last_valid_idx, rx0, rx1, tx0)

In [ ]:
ds.get_yaml_config()["emitter"]["motor_channel"]

In [ ]:
ds[1]["source_positions_at_t"][-1, 0]

In [ ]:
m = ds.get_m(ds.filenames[0])

In [ ]:
ds[1]["receiver_positions_at_t"][0].mean(axis=0)

In [ ]:
m[0, 0 : 0 + ds.snapshots_in_session * ds.step_size : ds.step_size].shape

In [ ]:
z = np.random.rand(10, 2)

In [ ]:
z[7:] = 0